In [5]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [6]:
df = pd.read_csv("dataset/googleplaystore.csv")

In [7]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
df.shape

(10841, 13)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [10]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [11]:
def value_to_float(x):
    # Convert the string feature into float/Integer
    #If there is 'K' or 'M' in the string, convert it to the corresponding number (1000 or 1000000)
    
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 10**3
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 10**6
        return 1000000.0
    
    # If the string cannot be converted, return 0 instead
    try:
        parsed_val = float(x)
    except ValueError:
        parsed_val = 0.0
    return parsed_val

In [12]:
df = df.loc[df["Rating"].notnull()]
df = df.loc[df["Rating"] <= 5]

df["Price"] = df["Price"].apply(lambda x: x.replace('$', ''))
df["Installs"] = df["Installs"].apply(lambda x: x.replace('+', ''))
df['Installs'] = df['Installs'].apply(lambda a: str(a).replace(',', '') if ',' in str(a) else a)
df['Installs'] = df['Installs'].apply(lambda a : int(a))
df['Reviews'] = df['Reviews'].apply(lambda a : int(a))

genres = df["Genres"].value_counts().head().index

str_cols = ["Size", "Price"]

for col in str_cols:
    df[[col]] = df[[col]].fillna(value="")
    df[col] = df[col].apply(value_to_float)

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [15]:
df.shape

(9360, 13)

In [16]:
df.describe()

,Rating,Reviews,Size,Installs,Price
count,9360.000000,9.360000e+03,9.360000e+03,9.360000e+03,9360.000000
mean,4.191838,5.143767e+05,1.894124e+07,1.790875e+07,0.961279
std,0.515263,3.145023e+06,2.302813e+07,9.126637e+07,15.821640
min,1.000000,1.000000e+00,0.000000e+00,1.000000e+00,0.000000
25%,4.000000,1.867500e+02,2.400000e+06,1.000000e+04,0.000000
50%,4.300000,5.955000e+03,9.500000e+06,5.000000e+05,0.000000
75%,4.500000,8.162750e+04,2.700000e+07,5.000000e+06,0.000000
max,5.000000,7.815831e+07,1.000000e+08,1.000000e+09,400.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9360 non-null   object 
 1   Category        9360 non-null   object 
 2   Rating          9360 non-null   float64
 3   Reviews         9360 non-null   int64  
 4   Size            9360 non-null   float64
 5   Installs        9360 non-null   int64  
 6   Type            9360 non-null   object 
 7   Price           9360 non-null   float64
 8   Content Rating  9360 non-null   object 
 9   Genres          9360 non-null   object 
 10  Last Updated    9360 non-null   object 
 11  Current Ver     9360 non-null   object 
 12  Android Ver     9360 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 1023.8+ KB


In [18]:
# There are 7 records where Reviews are greater than Installs 
df[df['Reviews'] > df['Installs']].shape

(7, 13)

In [19]:
# Dropping 7 records that have greater Reviews than Installs
df.drop(df[df['Reviews'] > df['Installs']].index,inplace=True)
df[df['Reviews'] > df['Installs']].shape

(0, 13)

In [20]:
# Dropping rows that have Price > 200
df.drop(df[df['Price'] > 200].index,inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9338 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9338 non-null   object 
 1   Category        9338 non-null   object 
 2   Rating          9338 non-null   float64
 3   Reviews         9338 non-null   int64  
 4   Size            9338 non-null   float64
 5   Installs        9338 non-null   int64  
 6   Type            9338 non-null   object 
 7   Price           9338 non-null   float64
 8   Content Rating  9338 non-null   object 
 9   Genres          9338 non-null   object 
 10  Last Updated    9338 non-null   object 
 11  Current Ver     9338 non-null   object 
 12  Android Ver     9338 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 1021.3+ KB


In [22]:
df = df[df['Category'] != '1.9']
value_counts = df['Category'].value_counts()
print(value_counts)

Category
FAMILY                 1742
GAME                   1093
TOOLS                   733
PRODUCTIVITY            351
MEDICAL                 349
COMMUNICATION           328
SPORTS                  319
FINANCE                 317
PHOTOGRAPHY             317
PERSONALIZATION         312
LIFESTYLE               308
BUSINESS                303
HEALTH_AND_FITNESS      297
SOCIAL                  258
SHOPPING                238
NEWS_AND_MAGAZINES      233
TRAVEL_AND_LOCAL        226
DATING                  195
BOOKS_AND_REFERENCE     178
VIDEO_PLAYERS           160
EDUCATION               155
ENTERTAINMENT           149
MAPS_AND_NAVIGATION     124
FOOD_AND_DRINK          109
HOUSE_AND_HOME           76
WEATHER                  75
AUTO_AND_VEHICLES        73
LIBRARIES_AND_DEMO       64
ART_AND_DESIGN           61
COMICS                   58
PARENTING                50
EVENTS                   45
BEAUTY                   42
Name: count, dtype: int64


In [23]:
category = pd.get_dummies(df['Category'], dtype='uint8')
df = pd.concat([df, category], axis=1)


In [24]:
df.drop('Category', axis=1, inplace=True)

In [25]:
df['Rating'].value_counts()

Rating
4.4    1107
4.3    1074
4.5    1037
4.2     951
4.6     823
4.1     705
4.0     565
4.7     499
3.9     386
3.8     299
5.0     268
3.7     239
4.8     233
3.6     172
3.5     162
3.4     128
3.3     102
4.9      87
3.0      83
3.1      69
3.2      63
2.9      44
2.8      42
2.7      25
2.6      25
2.5      21
2.3      20
2.4      19
1.0      16
2.2      14
1.9      13
2.0      12
1.7       8
1.8       8
2.1       8
1.6       4
1.4       3
1.5       3
1.2       1
Name: count, dtype: int64

In [26]:
print(df['Content Rating'].value_counts())

Content Rating
Everyone           7394
Teen               1082
Mature 17+          461
Everyone 10+        397
Adults only 18+       3
Unrated               1
Name: count, dtype: int64


In [27]:
df = df[df['Content Rating'] != 'Unrated']


In [28]:
content_rating = pd.get_dummies(df['Content Rating'], dtype='uint8')

df = pd.concat([df, content_rating], axis=1)

df.drop('Content Rating', axis=1, inplace=True)


In [29]:
df.head(6)

,App,Rating,Reviews,Size,Installs,Type,Price,Genres,Last Updated,Current Ver,...,SPORTS,TOOLS,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19000000.0,10000,Free,0.0,Art & Design,"January 7, 2018",1.0.0,...,0,0,0,0,0,0,1,0,0,0
1,Coloring book moana,3.9,967,14000000.0,500000,Free,0.0,Art & Design;Pretend Play,"January 15, 2018",2.0.0,...,0,0,0,0,0,0,1,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8700000.0,5000000,Free,0.0,Art & Design,"August 1, 2018",1.2.4,...,0,0,0,0,0,0,1,0,0,0
3,Sketch - Draw & Paint,4.5,215644,25000000.0,50000000,Free,0.0,Art & Design,"June 8, 2018",Varies with device,...,0,0,0,0,0,0,0,0,0,1
4,Pixel Draw - Number Art Coloring Book,4.3,967,2800000.0,100000,Free,0.0,Art & Design;Creativity,"June 20, 2018",1.1,...,0,0,0,0,0,0,1,0,0,0
5,Paper flowers instructions,4.4,167,5600000.0,50000,Free,0.0,Art & Design,"March 26, 2017",1.0,...,0,0,0,0,0,0,1,0,0,0


In [30]:
df = df.drop(['Last Updated', 'Current Ver', 'Android Ver','Type','App','Reviews', 'Installs','Genres'], axis=1)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9337 entries, 0 to 10840
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rating               9337 non-null   float64
 1   Size                 9337 non-null   float64
 2   Price                9337 non-null   float64
 3   ART_AND_DESIGN       9337 non-null   uint8  
 4   AUTO_AND_VEHICLES    9337 non-null   uint8  
 5   BEAUTY               9337 non-null   uint8  
 6   BOOKS_AND_REFERENCE  9337 non-null   uint8  
 7   BUSINESS             9337 non-null   uint8  
 8   COMICS               9337 non-null   uint8  
 9   COMMUNICATION        9337 non-null   uint8  
 10  DATING               9337 non-null   uint8  
 11  EDUCATION            9337 non-null   uint8  
 12  ENTERTAINMENT        9337 non-null   uint8  
 13  EVENTS               9337 non-null   uint8  
 14  FAMILY               9337 non-null   uint8  
 15  FINANCE              9337 non-null   uint8

In [32]:
df['Size'].value_counts()

Size
0.0           1892
14000000.0     165
12000000.0     161
15000000.0     159
11000000.0     159
              ... 
65000000.0       9
89000000.0       9
86000000.0       8
90000000.0       5
1000000.0        4
Name: count, Length: 182, dtype: int64

In [33]:
df = df[df['Size'] != 0]

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7445 entries, 0 to 10840
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rating               7445 non-null   float64
 1   Size                 7445 non-null   float64
 2   Price                7445 non-null   float64
 3   ART_AND_DESIGN       7445 non-null   uint8  
 4   AUTO_AND_VEHICLES    7445 non-null   uint8  
 5   BEAUTY               7445 non-null   uint8  
 6   BOOKS_AND_REFERENCE  7445 non-null   uint8  
 7   BUSINESS             7445 non-null   uint8  
 8   COMICS               7445 non-null   uint8  
 9   COMMUNICATION        7445 non-null   uint8  
 10  DATING               7445 non-null   uint8  
 11  EDUCATION            7445 non-null   uint8  
 12  ENTERTAINMENT        7445 non-null   uint8  
 13  EVENTS               7445 non-null   uint8  
 14  FAMILY               7445 non-null   uint8  
 15  FINANCE              7445 non-null   uint8

In [35]:
y = df[['Rating']]
x = df.drop('Rating',axis=1)

In [36]:
x.shape, y.shape

((7445, 40), (7445, 1))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [38]:
# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# # Make predictions on the test data
# Y_pred = rf.predict(X_test)

# mse = mean_squared_error(y_test, Y_pred)
# print("Mean squared error: ", mse)
# print(Y_pred)

In [ ]:
import random
from deap import base, creator, tools, algorithms

# Create a fitness function that measures the performance of the model
def evaluate(individual):
    print(individual)
    # Convert the individual into a list of hyperparameters
    n_estimators = individual[0]
    max_depth = individual[1]
    max_features = individual[2]
    min_samples_split = individual[3]
    min_samples_leaf = individual[4]
    
    # Create a random forest model with the given hyperparameters
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,random_state=42)
    
    # Train the model on the training data
    rf.fit(X_train, y_train)
    
    # Evaluate the model on the test data
    Y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, Y_pred)
    print(mse)
    # Return the mean squared error as the fitness value
    return mse,

# Define the genetic algorithm parameters
toolbox = base.Toolbox()
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("attr_n_estimators", random.randint, 1, 1000)
toolbox.register("attr_max_depth", random.randint, 1, 50)
toolbox.register("attr_max_features", random.randint, 1, len(x.columns))
toolbox.register("attr_min_samples_split", random.randint, 2,50)
toolbox.register("attr_min_samples_leaf", random.randint, 1,50)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_n_estimators, toolbox.attr_max_depth, toolbox.attr_max_features, toolbox.attr_min_samples_split, toolbox.attr_min_samples_leaf), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=len(x.columns), indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run the genetic algorithm
population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10)

# Select the best individual
best_individual = tools.selBest(population, k=1)[0]
n_estimators, max_depth, max_features = best_individual

# Train the model with the best hyperparameters
rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, random_state=42)
rf.fit(X_train, y_train)
Y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, Y_pred)
print("Mean squared error with optimized hyperparameters: ", mse)

/Users/user/Desktop/predicode/fastapi-backend/my_venv/lib/python3.11/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/user/Desktop/predicode/fastapi-backend/my_venv/lib/python3.11/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


[799, 11, 34, 3, 43]
0.3024326950545274
[707, 30, 11, 15, 3]
0.30011967584969285
[485, 7, 30, 42, 12]
0.3011920362888966
[904, 3, 11, 45, 39]
0.3067562730381874
[583, 16, 30, 39, 27]
0.3011617462216149
[258, 45, 9, 50, 25]
0.30042603537144574
[898, 25, 20, 42, 23]
0.3006484683481864
[560, 9, 17, 21, 36]
0.3021786923619354
[93, 28, 7, 15, 34]
0.3012270681067265
[207, 36, 27, 5, 25]
0.3009173260374966
[588, 20, 25, 2, 15]
0.29966129525187246
[885, 2, 11, 32, 31]
0.3081040595059811
[4, 3, 5, 47, 45]
0.306939447142773
[407, 6, 23, 36, 32]
0.30273214288575395
[948, 19, 10, 24, 3]
0.2985967123379218
[754, 8, 31, 27, 27]
0.30200645577436774
[742, 39, 12, 13, 44]
0.30190392031024926
[852, 23, 31, 24, 23]
0.3010898099245838
[349, 1, 19, 36, 24]
0.3096558473132222
[277, 33, 1, 45, 38]
0.3108451880814188
[47, 38, 29, 30, 30]
0.3014904512730525
[74, 19, 33, 43, 13]
0.29989867036210466
[826, 19, 4, 7, 14]
0.30018492260159163
[52, 23, 33, 35, 27]
0.3012052968925766
[490, 43, 3, 12, 14]
0.29999387693

In [40]:
# import random
# import numpy as np
# import pygad
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# import warnings
# from sklearn.exceptions import DataConversionWarning
# warnings.filterwarnings("ignore", category=DataConversionWarning)


In [41]:
# # Define the fitness function
# def fitness_func(self, solution, solution_idx):
#     if type(solution) == list:
#         solution = np.array(solution)
#     n_estimators = int(solution[0])
#     max_depth = int(solution[1])
#     max_features = int(solution[2])
    
#     # Create a random forest model with the given hyperparameters
#     rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, random_state=42)
    
#     # Train the model on the training data
#     rf.fit(X_train, y_train)
    
#     # Evaluate the model on the test data
#     Y_pred = rf.predict(X_test)
#     mse = mean_squared_error(y_test, Y_pred)

#     print("mse: ", mse)
#     # Return the mean squared error as the fitness value
#     fitness = 1.0 / (mse + 1e-6)
#     print("fitness: ",fitness)
#     return fitness

# # Define the initial population
# num_generations = 10
# num_parents_mating = 10
# sol_per_pop = 50
# num_genes = 3
# parent_selection_type = "rank"
# crossover_type = "single_point"
# mutation_type = "random"

# init_range_low = 1
# init_range_high = 1000
# gene_space = [(init_range_low, init_range_high), (1, 50), (1, len(x.columns))]

# # initial_population = pygad.initial_population(population_size=sol_per_pop, 
# #                                               num_genes=num_genes, 
# #                                               gene_space=gene_space)

# initial_population = np.random.randint(low=init_range_low, high=init_range_high+1, size=(sol_per_pop, num_genes))


# # Create the genetic algorithm
# ga_instance = pygad.GA(num_generations=num_generations,
#                         num_parents_mating=num_parents_mating,
#                         sol_per_pop=sol_per_pop,
#                         num_genes=num_genes,
#                         gene_space=gene_space,
#                         parent_selection_type=parent_selection_type,
#                         crossover_type=crossover_type,
#                         mutation_type=mutation_type,
#                         initial_population=initial_population,
#                         fitness_func=fitness_func)

# # Run the genetic algorithm
# ga_instance.run()

# # Get the best solution found by the genetic algorithm
# best_solution, best_solution_fitness = ga_instance.best_solution()

# # Train the model with the best hyperparameters
# n_estimators, max_depth, max_features = solution
# rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, random_state=42)
# rf.fit(X_train, y_train)
# Y_pred = rf.predict(X_test)
# mse = mean_squared_error(y_test, Y_pred)
# print("Mean squared error with optimized hyperparameters: ", mse)

/Users/user/Desktop/predicode/fastapi-backend/my_venv/lib/python3.11/site-packages/pygad/pygad.py:623: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  if not self.suppress_warnings: warnings.warn("The percentage of genes to mutate (mutation_percent_genes={mutation_percent}) resutled in selecting ({mutation_num}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.".format(mutation_percent=mutation_percent_genes, mutation_num=mutation_num_genes))


mse:  0.345109834007022
fitness:  2.8976198411077787
mse:  0.3454644193055369
fitness:  2.894645727523828
mse:  0.34462112162595093
fitness:  2.9017289873381635
mse:  0.3451068721599962
fitness:  2.897644709583408
mse:  0.3449978096266962
fitness:  2.8985607257081374
mse:  0.30750835595319315
fitness:  3.2519335774363003
mse:  0.34612462391180043
fitness:  2.889124441866863
mse:  0.3450144090507251
fitness:  2.898421269796032
mse:  0.3463317199302907
fitness:  2.8873968367796103
mse:  0.34506858639895077
fitness:  2.8979662057027946
mse:  0.3538215699699765
fitness:  2.826275328012158
mse:  0.3448257327063691
fitness:  2.9000071779572023
mse:  0.3454990862300261
fitness:  2.8943552834143804
mse:  0.3469941210631801
fitness:  2.881884900675368
mse:  0.34516239321078723
fitness:  2.897178610679933
mse:  0.3448344377952171
fitness:  2.8999339696457094
mse:  0.34509990871952434
fitness:  2.8977031782107976
mse:  0.3448786196118395
fitness:  2.899562465087081
mse:  0.34877067770827186
fitne

ValueError: too many values to unpack (expected 2)